In [1]:
import torch
from torch import nn
from torch.utils.data import DataLoader
from torch.utils.tensorboard import SummaryWriter
from tqdm import tqdm

from Dataset import Dataset

writer = SummaryWriter()

In [2]:
# const
batch_size = 32

In [3]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using {device} device")

Using cuda device


In [4]:
train_dataset = Dataset("Train")
train_dataloader = DataLoader(train_dataset, batch_size=batch_size)

test_dataset = Dataset("Test")
test_dataloader = DataLoader(test_dataset, batch_size=batch_size)

(41, 17473, 48)
(41, 8737, 48)


In [5]:
class MLP(nn.Module):
    def __init__(self, input_size, hidden_size=64, out_features=64):
        super(MLP, self).__init__()
        self.flatten = nn.Flatten()
        self.linear = nn.Linear(input_size, hidden_size)
        self.relu = nn.ReLU()
        self.linear2 = nn.Linear(hidden_size, out_features)
        
    def forward(self, x):
        x = self.flatten(x)
        x = self.linear(x)
        x = self.relu(x)
        x = self.linear2(x)
        return x

In [6]:
model = MLP(41*24, 64, 41*24).to(device)

In [7]:
loss_fn = nn.MSELoss(reduction='mean')
optimizer = torch.optim.Adam(model.parameters(), lr=1e-3)

In [8]:
def train(dataloader, model, loss_fn, optimizer):
    size = len(dataloader.dataset)
    model.train()
    for batch, data in enumerate(tqdm(dataloader)):
        input_data = data[:, :, :24].to(device).to(torch.float32)
        ground_truth = data[:, :, 24:].to(device).to(torch.float32)
        
        pred = model(input_data).view(-1, 41, 24)
        loss = loss_fn(pred, ground_truth)
        writer.add_scalar('Loss/train', loss, batch)
        
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        
        if batch % 100 == 0:
            loss, current = loss.item(), batch * len(input_data)
            print(f"loss: {loss:>7f}  [{current:>5d}/{size:>5d}]")

In [9]:
def test(dataloader, model, loss_fn):
    test_loss = 0
    model.eval()
    with torch.no_grad():
        for batch, data in enumerate(tqdm(dataloader)):
            input_data = data[:, :, :24].to(device).to(torch.float32)
            ground_truth = data[:, :, 24:].to(device).to(torch.float32)
            
            pred = model(input_data).view(-1, 41, 24)
            loss = loss_fn(pred, ground_truth)
            test_loss += loss

    test_loss /= len(dataloader)
    print(f"Avg loss: {test_loss:>8f} \n")

In [10]:
# epochs = 10
# for t in range(epochs):
#     print(f"Epoch {t+1}\n-------------------------------")
#     train(train_dataloader, model, loss_fn, optimizer)
#     test(test_dataloader, model, loss_fn)
# print("Done!")

In [11]:
writer.close()

In [14]:
from LineNet import LineNet

linenet = LineNet(64).to(device)
for batch, data in enumerate(tqdm(train_dataloader)):
    input_data = data[:, :, :24].to(device).to(torch.float32)
    ground_truth = data[:, :, 24:].to(device).to(torch.float32)
    print(input_data.shape)
    print(ground_truth.shape)
#     print(linenet(input_data, batch_size, device).shape)
    break

  0%|          | 0/547 [00:00<?, ?it/s]

torch.Size([32, 41, 24])
torch.Size([32, 41, 24])
